## Topics

- Runnable
- Prompt Template
- Intro to LangGraph and memory
- Output parsing

In [ ]:
# %pip install langchain langchain-openai
# !pip --version
# !pip install grandalf

In [ ]:
# MAKE SURE LANGCHAIN VERSION IS 0.2.17

import langchain
print(langchain.__version__)

In [ ]:
import os
from langchain_openai import ChatOpenAI  # pip install langchain-openai

llm = ChatOpenAI(
    openai_api_key="HERE YOUR API KEY", 
    temperature=.75, 
    max_tokens=1024, 
    request_timeout=30
)

In [ ]:
llm.invoke("Hello!")

In [ ]:
from langchain.prompts import ChatPromptTemplate  # pip install langchain

prompt = ChatPromptTemplate.from_messages([
    ("system", "Act as a world class Machine Learning engineer. Use english language. End your answers with a reference to the beauty of using data science in any decision you make."),
    ("user", "{input}")
])

# concatenazione del prompt al modello
chain = prompt | llm

## Interfaccia Runnable

Per semplificare la creazione di catene di eventi/esecuzioni anche molto complesse, tutti i componenti di LangChain implementano un protocollo "runnable" attraverso un'interfaccia comune che consente l'utilizzo standard di qualsiasi componente. Di seguito sono riportati i tre metodi principali:

* **stream** - invia risposte parziali man mano che vengono generate
* **invoke** - esegue la catena su un singolo input
* **batch** - esegue la catena su più input

### Input e Output dei Componenti Principali
<img src="assets/componenti_io.png" width="600">

Uno dei vantaggi delle interfacce Runnable è che i componenti eseguibili possono essere collegati insieme in sequenze di esecuzione, permettendo all'output di un componente di diventare automaticamente l'input di un altro. Il comando *pipe* **|** è utilizzato a questo scopo nel LCEL (LangChain Expression Language), consentendo la creazione di componenti eseguibili da altri componenti eseguibili configurandoli in una sequenza che lavorerà in modo sinergico.


In [ ]:
chain.invoke({"input": "hello!"})

In [ ]:
chain.get_graph().print_ascii()

### Flusso di Lavoro per Conversazioni AI con LangChain e LangGraph

Questo codice definisce un flusso di lavoro per gestire conversazioni AI utilizzando le librerie **LangChain** e **LangGraph**.

#### Creazione del Grafo di Stato
Si crea un oggetto `StateGraph` basato su uno schema di stato dei messaggi, che rappresenta il flusso di esecuzione:

```python
workflow = StateGraph(state_schema=MessagesState)
def call_model(state: MessagesState):
    ...

```
#### Funzione per Chiamare il Modello
La funzione `call_model`  prepara un prompt iniziale e aggiunge i messaggi provenienti dallo stato. Questi messaggi vengono poi passati al modello per ottenere una risposta.

#### Aggiunta di Nodi e Collegamenti
Viene aggiunto un nodo chiamato "model" al flusso di lavoro e viene creato un collegamento di esecuzione dal nodo di inizio (`START`) al nodo "model"

#### Checkpointer in Memoria
Per salvare e recuperare lo stato della conversazione, viene aggiunto un oggetto `MemorySaver` come `checkpointer`

#### Compilazione del Flusso di Lavoro
Il flusso di lavoro viene compilato per diventare eseguibile. 

Questo approccio consente di creare flussi di lavoro modulari per la gestione delle conversazioni AI, con nodi eseguibili e collegamenti che facilitano la gestione dei messaggi e la memoria delle sessioni.

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    system_prompt = (
        "You are a helpful assistant. "
        "Answer all questions to the best of your ability."
    )
    messages = [SystemMessage(content=system_prompt)] + state["messages"]
    response = chain.invoke(messages)
    return {"messages": response}


# Define the node and edge
workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

# Add simple in-memory checkpointer
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
resp = app.invoke(
    {"messages": [HumanMessage(content="Translate to French: I love programming.")]},
    config={"configurable": {"thread_id": "1"}},
)

In [ ]:
resp['messages'][-1].content

In [ ]:
resp = app.invoke({
    "messages": [HumanMessage(content="what did i just say?")]},
    config={"configurable": {"thread_id": "1"}})

In [ ]:
for r in resp['messages']:
    print(type(r).__name__, r.content)
    print("\n")

# Introduzione a PromptTemplate

Il `PromptTemplate` è una funzionalità potente progettata per semplificare e standardizzare la creazione di prompt per varie applicazioni, come chatbot, risposte automatiche o moduli di inserimento dati. Fornisce un formato strutturato che può essere riutilizzato in diversi scenari, garantendo coerenza ed efficienza nel modo in cui vengono richiesti e gestiti gli input.


In [ ]:
# dynamic template and use of a Memory Buffer

template = """Act as a data scientist answering to every question with references to the beauty of Data Science.
New question: {question}
Answer:"""

from langchain_core.prompts.prompt import PromptTemplate

prompt = PromptTemplate(template=template)


from langchain.chains import LLMChain

conversation = prompt | llm 

In [ ]:
conversation.invoke({"question": "Hello, i like the orange color."})

## LLM output parsing

<a href="https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/quick_start/" target="_blank">source</a>

Language models output text. But many times you may want to get more structured information than just text back. This is where output parsers come in.

**Output parsers** are classes that help *structure language model responses*. 

There are two main methods an output parser must implement:

- "Get format instructions": A method which returns a string containing instructions for how the output of a language model should be formatted.
- "Parse": A method which takes in a string (assumed to be the response from a language model) and parses it into some structure.

And then one optional one:

- "Parse with prompt": A method which takes in a string (assumed to be the response from a language model) and a prompt (assumed to be the prompt that generated such a response) and parses it into some structure. The prompt is largely provided in the event the OutputParser wants to retry or fix the output in some way, and needs information from the prompt to do so.

In [ ]:
# OutputParser

template = """Act as a data scientist answering to every question with references to the beauty of Data Science.
New question: {question}
Answer:"""

prompt = PromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

chain.invoke({"question": "Mi piace il colore arancione"})

In [ ]:
from pydantic import BaseModel, Field, validator

class User(BaseModel):
    id: int = Field(description="user identification number")
    name: str = Field(description="user name")
    mail: str = Field(description="user mail address")
    

from langchain.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=User)

print(parser.get_format_instructions())

In [ ]:
prompt = PromptTemplate(
    template="Analyze this text\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

prompt

In [ ]:
chain = prompt | llm | parser

query = "id:1, name: John Doe, e-mail: john.doe@johndoe.com"

chain.invoke({"query": query})

In [ ]:
chain = prompt | llm | parser

query = "my name is giuseppe mastrandrea, my email is mastrandreagiuseppe@gmail.com and my id is 500"

chain.invoke({"query": query})

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser(pydantic_object=User)

chain = prompt | llm | json_parser

chain.invoke({"query": query})